In [27]:
import numpy as np
from collections import defaultdict

In [149]:
class PuzzleSolver:
    
    def __init__(self, size, matrix_values=None):
        
        # Considering tile 8 to be the blank tile.
        if np.all(matrix_values==None):
            self.puzzle = np.arange(0, size**2)
        else:
            self.puzzle = matrix_values
        self.puzzle = self.puzzle.reshape(size, size) # Reshape to n*n else:
        
        # Init a dict of visited states
        self.visited_states = {}
        
        # Init the dfs state dictionary
        self.dfs_tree_dict = defaultdict(list)
        return
    
    
    def print_puzzle(self):
        clean_puzzle = np.array(self.puzzle.copy(), dtype='str')
        clean_puzzle[clean_puzzle=='8'] = " "
        print(clean_puzzle, "\n")
        return
    
    
    def heuristic_cost_per_cell(self, rc, cc):
        # This is essentially manhattan distance, d = |x - x_org| + |y - y_org|
        return rc + cc;
    
    
    def binary_heuristic_cost(self, configuration):
        # All tiles at incorrect positions add to the cost.
        cost = 0
        for _, i in enumerate(list(configuration.ravel())):
            if i!=8 and _ != i:
                cost += 1
        return cost
    
    
    def total_heuristic_cost(self, configuration):
        """
        This is the sum of cost of each tile. Cost of each tile is how far it is from the correct row and the column.
        """
        
        total_cost = 0
        row_wise_cost = {0: 0, 1: 0, 2:0}
        
        for i in range(0, 3):
            for j in range(0, 3):
# #                 row_wise_cost[i] += self.binary_heuristic_cost()
                if configuration[i][j] != 8:
                    total_cost += self.heuristic_cost_per_cell(self.row_cost(configuration[i][j], i), self.col_cost(configuration[i][j], j))
                
#         if row_wise_cost[0] > 0:
#             return row_wise_cost[0]
        total_cost += self.binary_heuristic_cost(configuration)
        return int(total_cost)

    
    def row_cost(self, tile_num, x_curr):
        """
        row_cost = distance between the row of the final position and the current row index
        """
        # tile_num can give us the ideal position for that tile
        if tile_num == 8:
            return 0
        #print(tile_num, x_curr, abs(x_curr - (tile_num)//3))
        return abs(x_curr - (tile_num)//3)
    
    
    def col_cost(self, tile_num, y_curr):
        """
        col_cost = distance between the column of the final position and the current column index
        """
        # tile_num can give us the ideal position for htat tile
        if tile_num == 8:
            return 0
        #print(tile_num, y_curr, abs(y_curr - (tile_num)%3))
        return abs(y_curr - (tile_num)%3)

    
    def get_possible_moves(self):
        """
        Returns possible move based on where the blank tile is located. Only tiles adjacent to the blank tile can be moved.
        """
        
        # finding pos of blank tile (i.e. 8)
        rblank, cblank = np.where(self.puzzle==8)[0][0], np.where(self.puzzle==8)[1][0]
        
        possible_swap_locs = [(rblank, cblank+j) for j in [-1, 1] if cblank+j in [0, 1, 2]] + [(rblank + i, cblank) for i in [-1, 1] if rblank+i in [0, 1, 2]]
        return possible_swap_locs
    
    
    def is_visited_state(self, config):
        rep = np.array_str(config.ravel())
        if rep in self.visited_states:
            return True
        return False
    
    
    def add_visited_state(self, config):
        rep = np.array_str(config.ravel())
        self.visited_states[rep] = True
#         print("visited: ", rep)
        return
    
    
    def backtrack(self, dfs_level):
        if dfs_level < 0:
            return None
        
        for config in self.dfs_tree_dict[dfs_level]:
            if not self.is_visited_state(config):
                return config
        
        return self.backtrack(dfs_level-1)
        
    
    def perform_least_swap(self, possible_moves, prev_cost, dfs_level):
        """
        Perform the swap that has the least heuristic cost.
        """
        
        move_costs = {}
        move_config = {}

        for _, move in enumerate(possible_moves):
            configuration = self.puzzle.copy()
            configuration[np.where(configuration==8)[0][0], np.where(configuration==8)[1][0]] = configuration[move[0], move[1]]
            configuration[move[0], move[1]] = 8
            move_costs[_] = self.total_heuristic_cost(configuration)
            move_config[_] = configuration
            
            # Add to the dfs tree
            self.dfs_tree_dict[dfs_level].append(configuration)
            
        
        min_cost = np.inf
        min_candidates = []
        min_candidates = sorted(move_costs.items(), key=lambda x: x[1])
                
        min_cost = min_candidates[0][1]
        print("Prev cost: {} \t Current min cost: {}".format(prev_cost, min_cost))
        
        #if min_cost == prev_cost or self.is_visited_state(move_config[min_candidates[0][0]]):
        if self.is_visited_state(move_config[min_candidates[0][0]]):
            found_alt_move = False
            for iterid in range(1, len(min_candidates)):
                #if min_cost < min_candidates[iterid][1] and not self.is_visited_state(move_config[min_candidates[iterid][0]]):
                if not self.is_visited_state(move_config[min_candidates[iterid][0]]):
                    found_alt_move = True
                    self.puzzle = move_config[min_candidates[iterid][0]]
                    self.add_visited_state(move_config[min_candidates[iterid][0]])
                    min_cost = min_candidates[iterid][1]
                    break
            if not found_alt_move:
                print("Next move not found. Will have to backtrack.")
                prev_backtrack_config = self.backtrack(dfs_level)
                if np.all(prev_backtrack_config==None):
                    print("Exhausted all states. Unsolvable! Exiting!")
                    sys.exit(0)
                self.puzzle = prev_backtrack_config
                self.add_visited_state(prev_backtrack_config)
        else:
            self.puzzle = move_config[min_candidates[0][0]]
            self.add_visited_state(move_config[min_candidates[0][0]])
        
#         if len(min_candidates) > 1:
#             print("Multiple min candidates:")
#             for i in min_candidates:
#                 print(i)
        return
    
    
    def search_paths(self):
        iter = 0
        dfs_level = 0
        
        print("Initial State")
        self.print_puzzle()
        print("===========================================================================")
        
        while self.total_heuristic_cost(self.puzzle) > 0:
            prev_cost = self.total_heuristic_cost(self.puzzle)
            
            print("Iteration {}:".format(iter))
            possible_moves = self.get_possible_moves()
            self.perform_least_swap(possible_moves, prev_cost, dfs_level)
            
            #print("Iteration {} (Current cost: {})".format(iter, self.total_heuristic_cost(self.puzzle)))
            self.print_puzzle()
            
            iter += 1
            dfs_level += 1 
            print("===========================================================================")
        
        return

In [150]:
ps = PuzzleSolver(3, np.array([0,1,2,8,3,4,6,7,5]))
ps.search_paths()

Initial State
[['0' '1' '2']
 [' ' '3' '4']
 ['6' '7' '5']] 

Iteration 0:
Prev cost: 6 	 Current min cost: 4
[['0' '1' '2']
 ['3' ' ' '4']
 ['6' '7' '5']] 

Iteration 1:
Prev cost: 4 	 Current min cost: 2
[['0' '1' '2']
 ['3' '4' ' ']
 ['6' '7' '5']] 

Iteration 2:
Prev cost: 2 	 Current min cost: 0
[['0' '1' '2']
 ['3' '4' '5']
 ['6' '7' ' ']] 

